# 문제 11 유형
- DataSet_11.csv 이용

In [1]:
import pandas as pd

df11_orig = pd.read_csv('datasets/Dataset_11.csv', encoding='utf8')
df11 = df11_orig.copy()
df11.head(3)

,Country,Happiness_Rank,Happiness_Score,year
0,Switzerland,1,7.587,2015
1,Iceland,2,7.561,2015
2,Denmark,3,7.527,2015


## 1. 3년 연속 데이터가 기록되지 않은 국가 개수
- 해당 국가의 데이터는 제거 후 2번 문제부터 풀이

In [3]:
year = df11['year'].unique()
year

array([2015, 2016, 2017], dtype=int64)

In [4]:
df11_pivot = pd.pivot_table(data=df11, index='Country', columns='year')
df11_pivot

Happiness_Rank               Happiness_Score              
year                  2015   2016   2017            2015   2016   2017
Country                                                               
Afghanistan          153.0  154.0  141.0           3.575  3.360  3.794
Albania               95.0  109.0  109.0           4.959  4.655  4.644
Algeria               68.0   38.0   53.0           5.605  6.355  5.872
Angola               137.0  141.0  140.0           4.033  3.866  3.795
Argentina             30.0   26.0   24.0           6.574  6.650  6.599
...                    ...    ...    ...             ...    ...    ...
Venezuela             23.0   44.0   82.0           6.810  6.084  5.250
Vietnam               75.0   96.0   94.0           5.360  5.061  5.074
Yemen                136.0  147.0  146.0           4.077  3.724  3.593
Zambia                85.0  106.0  116.0           5.129  4.795  4.514
Zimbabwe             115.0  131.0  138.0           4.610  4.193  3.875

[166 rows x 6 columns]

In [15]:
df11_pivot.isnull().sum()

                 year
Happiness_Rank   2015     8
                 2016     9
                 2017    11
Happiness_Score  2015     8
                 2016     9
                 2017    11
dtype: int64

In [5]:
df11_dropna = df11_pivot.dropna(axis=0)
df11_dropna

Happiness_Rank               Happiness_Score              
year                  2015   2016   2017            2015   2016   2017
Country                                                               
Afghanistan          153.0  154.0  141.0           3.575  3.360  3.794
Albania               95.0  109.0  109.0           4.959  4.655  4.644
Algeria               68.0   38.0   53.0           5.605  6.355  5.872
Angola               137.0  141.0  140.0           4.033  3.866  3.795
Argentina             30.0   26.0   24.0           6.574  6.650  6.599
...                    ...    ...    ...             ...    ...    ...
Venezuela             23.0   44.0   82.0           6.810  6.084  5.250
Vietnam               75.0   96.0   94.0           5.360  5.061  5.074
Yemen                136.0  147.0  146.0           4.077  3.724  3.593
Zambia                85.0  106.0  116.0           5.129  4.795  4.514
Zimbabwe             115.0  131.0  138.0           4.610  4.193  3.875

[146 rows x 6 columns]

### 답
20

## 2. 국가별 행복지수 증감률
- 증감률 = (2017년 행복지수 - 2015년 행복지수) / 2
- 행복지수 증감률이 가장 높은 3개 국가의 행복지수가 높은 순서대로

In [11]:
df11_hs = df11_dropna['Happiness_Score'].reset_index()
df11_hs

year,Country,2015,2016,2017
0,Afghanistan,3.575,3.360,3.794
1,Albania,4.959,4.655,4.644
2,Algeria,5.605,6.355,5.872
3,Angola,4.033,3.866,3.795
4,Argentina,6.574,6.650,6.599
...,...,...,...,...
141,Venezuela,6.810,6.084,5.250
142,Vietnam,5.360,5.061,5.074
143,Yemen,4.077,3.724,3.593
144,Zambia,5.129,4.795,4.514


In [13]:
df11_hs['rate'] = (df11_hs[2017] - df11_hs[2015]) / 2
df11_hs

year,Country,2015,2016,2017,rate
0,Afghanistan,3.575,3.360,3.794,0.1095
1,Albania,4.959,4.655,4.644,-0.1575
2,Algeria,5.605,6.355,5.872,0.1335
3,Angola,4.033,3.866,3.795,-0.1190
4,Argentina,6.574,6.650,6.599,0.0125
...,...,...,...,...,...
141,Venezuela,6.810,6.084,5.250,-0.7800
142,Vietnam,5.360,5.061,5.074,-0.1430
143,Yemen,4.077,3.724,3.593,-0.2420
144,Zambia,5.129,4.795,4.514,-0.3075


In [16]:
df11_hs.sort_values(by='rate', ascending=False)

year,Country,2015,2016,2017,rate
71,Latvia,5.098,5.560,5.850,0.3760
108,Romania,5.124,5.528,5.825,0.3505
129,Togo,2.839,3.303,3.495,0.3280
112,Senegal,3.904,4.219,4.535,0.3155
44,Gabon,3.896,4.121,4.465,0.2845
...,...,...,...,...,...
144,Zambia,5.129,4.795,4.514,-0.3075
145,Zimbabwe,4.610,4.193,3.875,-0.3675
51,Haiti,4.518,4.028,3.603,-0.4575
73,Liberia,4.571,3.622,3.533,-0.5190


### 답
Latvia, Romania, Togo

## 3. 년도별 행복지수 평균이 유의미하게 차이나는지
- 검정통계량 소수점 넷째 자리까지

In [22]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

df11_anova = df11[df11['Country'].isin(df11_dropna.index)]
df11_anova

,Country,Happiness_Rank,Happiness_Score,year
0,Switzerland,1,7.587,2015
1,Iceland,2,7.561,2015
2,Denmark,3,7.527,2015
3,Norway,4,7.522,2015
4,Canada,5,7.427,2015
...,...,...,...,...
464,Togo,150,3.495,2017
465,Rwanda,151,3.471,2017
466,Syria,152,3.462,2017
467,Tanzania,153,3.349,2017


In [23]:
# C(수치형 변수) : 범주형으로 인식
anova = ols('Happiness_Score ~ C(year)', df11_anova).fit()

In [25]:
anova_lm(anova)

,df,sum_sq,mean_sq,F,PR(>F)
C(year),2.0,0.011198,0.005599,0.004277,0.995732
Residual,435.0,569.472307,1.309132,NaN,NaN


### 답
0.0043

### 참고
- anova 분석 다른 방법

In [34]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

anova_result = pairwise_tukeyhsd(df11_anova['Happiness_Score'], df11_anova['year'])
print(anova_result)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
  2015   2016  -0.0112   0.9 -0.3261 0.3038  False
  2015   2017   -0.001   0.9 -0.3159  0.314  False
  2016   2017   0.0102   0.9 -0.3048 0.3251  False
--------------------------------------------------


In [37]:
from scipy.stats import f_oneway

f, p = f_oneway(df11_hs[2015], df11_hs[2016], df11_hs[2017])
print(f, p)

0.004276725037689305 0.9957324489944479


# 문제 12 유형
- DataSet_12.csv 이용

In [38]:
df12_orig = pd.read_csv('datasets/Dataset_12.csv', encoding='utf8')
df12 = df12_orig.copy()
df12.head(3)

,Age,Gender,Dependent_Count,Education_Level,is_Married,Read_Book_per_Year,Income_Range
0,52,M,2,석사,1,45,D
1,46,F,4,박사,1,26,X
2,53,F,0,석사,0,48,A


## 1. 수치형 변수를 대상으로 피어슨 상관분석
- 연간 독서량과 가장 상관관계가 강한 변수의 상관계수
- 반올림하여 소수점 셋째 자리까지

In [41]:
df12.corr()

,Age,Dependent_Count,is_Married,Read_Book_per_Year
Age,1.000000,-0.143767,0.040360,0.796843
Dependent_Count,-0.143767,1.000000,0.024472,-0.132591
is_Married,0.040360,0.024472,1.000000,0.027221
Read_Book_per_Year,0.796843,-0.132591,0.027221,1.000000


In [42]:
df12.corr()['Read_Book_per_Year']

Age                   0.796843
Dependent_Count      -0.132591
is_Married            0.027221
Read_Book_per_Year    1.000000
Name: Read_Book_per_Year, dtype: float64

### 답
0.797

## 2. 석사 이상 여부에 따른 연간 독서량 평균 차이
- 독립 2표본 t검정
- 등분산 가정
- p-value 반올림하여 소수점 셋째 자리까지

In [43]:
df12_석사 = df12[df12['Education_Level'] == '석사']['Read_Book_per_Year']
df12_n석사 = df12[df12['Education_Level'] != '석사']['Read_Book_per_Year']

In [44]:
from scipy.stats import ttest_ind

t, p = ttest_ind(df12_석사, df12_n석사, equal_var=True)
print(t, p)

0.09821366362358962 0.9217665713124468


### 답
0.922

## 3. 독서량의 다중선형회귀분석
- 수치형 자료를 독립변수
- 독립변수가 고정이면서 나이만 다를 때, 40살은 30살보다 독서량이 얼마나 많은지
- 반올림하여 정수로 표기

In [45]:
from sklearn.linear_model import LinearRegression

X = df12[['Age', 'Dependent_Count', 'is_Married']]
y = df12['Read_Book_per_Year']

lr = LinearRegression()
lr.fit(X, y)
print(lr.coef_)

[ 0.78944324 -0.11496195 -0.07190099]


In [47]:
lr.coef_[0] * 10

7.894432436609117

### 답
8

### 참고

In [48]:
var_list = df12.columns[df12.dtypes != 'object'].drop('Read_Book_per_Year')
var_list

Index(['Age', 'Dependent_Count', 'is_Married'], dtype='object')

In [49]:
from statsmodels.formula.api import ols

formula = 'Read_Book_per_Year~' + '+'.join(var_list)

result12 = ols(formula, df12).fit()
result12.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     Read_Book_per_Year   R-squared:                       0.635
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                     2901.
Date:                Tue, 24 Aug 2021   Prob (F-statistic):               0.00
Time:                        15:59:23   Log-Likelihood:                -15057.
No. Observations:                5000   AIC:                         3.012e+04
Df Residuals:                    4996   BIC:                         3.015e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.3231      0.445     -0.726      0.468      -1.195       0.549
Age                 0.7894      0.009     91.923      0.000       0.773       0.806
Dependent_Count    -0.1150      0.054     -2.116      0.034      -0.221      -0.008
is_Married         -0.0719      0.140     -0.514      0.608      -0.346       0.203
==============================================================================
Omnibus:                       72.985   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               76.307
Skew:                          -0.288   Prob(JB):                     2.69e-17
Kurtosis:                       3.183   Cond. No.                         303.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# 문제 13 유형
- DataSet13_train.csv / DataSet13_test.csv 이용

In [50]:
df13_train = pd.read_csv('datasets/Dataset_13_train.csv', encoding='utf8')
df13_test = pd.read_csv('datasets/Dataset_13_test.csv', encoding='utf8')

df13_train.head(3)

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,last_new_job,training_hours,target
0,0.516,Male,1,0,Graduate,STEM,13,5,36,1
1,0.766,Male,1,0,Graduate,STEM,12,5,264,0
2,0.920,Male,1,0,Masters,STEM,10,2,99,0


## 1. 남녀별 상관계수
- train 데이터 이용
- 경력과 최근 이직 시 공백기간
- 더 높은 상관계수를 반올림하여 소수점 둘째 자리까지

In [52]:
df13_train_male = df13_train[df13_train['gender']=='Male'][['experience', 'last_new_job']]
df13_train_female = df13_train[df13_train['gender']=='Female'][['experience', 'last_new_job']]

In [53]:
df13_train_male.corr()

,experience,last_new_job
experience,1.000000,0.411155
last_new_job,0.411155,1.000000


In [54]:
df13_train_female.corr()

,experience,last_new_job
experience,1.000000,0.451898
last_new_job,0.451898,1.000000


### 답
0.45

### 참고

In [82]:
df13_train.groupby(['gender'])[['experience', 'last_new_job']].corr()

experience  last_new_job
gender                                       
Female experience      1.000000      0.451898
       last_new_job    0.451898      1.000000
Male   experience      1.000000      0.411155
       last_new_job    0.411155      1.000000

## 2. 데이터 분석 관력 직무 경험과 이직 의사의 독립성 검정
- train 데이터
- STEM 전공자 대상
- 도시 개발 지수가 85백분위수 초과에 거주하는 사람 대상
- 이직 의사 여부(target)은 문자열로 변경
- p-value 반올림하여 소수점 둘째 자리까지

In [57]:
quantile_85 = df13_train['city_development_index'].quantile(0.85)
quantile_85

0.9200000000000002

In [58]:
df13_chi2 = df13_train[(df13_train['major_discipline']=='STEM') & 
                       (df13_train['city_development_index'] > quantile_85)]

In [60]:
df13_chi2['target'] = df13_chi2['target'].astype(str)
df13_chi2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145 entries, 6 to 1480
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_development_index  145 non-null    float64
 1   gender                  145 non-null    object 
 2   relevent_experience     145 non-null    int64  
 3   enrolled_university     145 non-null    int64  
 4   education_level         145 non-null    object 
 5   major_discipline        145 non-null    object 
 6   experience              145 non-null    int64  
 7   last_new_job            145 non-null    int64  
 8   training_hours          145 non-null    int64  
 9   target                  145 non-null    object 
dtypes: float64(1), int64(5), object(4)
memory usage: 12.5+ KB


C:\Users\master\anaconda3\envs\multi\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [70]:
df13_pivot = pd.pivot_table(data=df13_chi2, columns='target',
                           index='relevent_experience', aggfunc='count',
                           values='education_level')

In [71]:
df13_pivot

target,0,1
relevent_experience,,
0,30,7
1,93,15


In [72]:
from scipy.stats import chi2_contingency

chi2, p, _, _ = chi2_contingency(df13_pivot)
print(chi2, p)

0.22142211039568746 0.6379584714909265


### 답
0.64

### 참고

In [84]:
df13_crosstab = pd.crosstab(df13_chi2['relevent_experience'], df13_chi2['target'])

chi2, p, _, _ = chi2_contingency(df13_crosstab)
print(chi2, p)

0.22142211039568746 0.6379584714909265


## 3. 어떤 직원이 이직 의사를 가지고 있는지 파악
- 의사결정나무
- target을 종속변수, string이 아닌 변수를 독립 변수
- 평가는 test
- 정확도는 반올림하여 소수점 둘째 자리까지

In [76]:
from sklearn.tree import DecisionTreeClassifier

features = df13_train.columns[df13_train.dtypes != 'object'].drop('target')

In [77]:
features

Index(['city_development_index', 'relevent_experience', 'enrolled_university',
       'experience', 'last_new_job', 'training_hours'],
      dtype='object')

In [78]:
X_train = df13_train[features]
y_train = df13_train['target']
X_test = df13_test[features]
y_test = df13_test['target']

In [79]:
dt_clf = DecisionTreeClassifier(random_state=123)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)

In [80]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, pred)
print('정확도 :', acc)

정확도 : 0.672


### 답
0.67

# 문제 14 유형
- DataSet_14.csv 이용

In [87]:
df14 = pd.read_csv('datasets/Dataset_14.csv', encoding='utf8')
df14.head(3)

,id,published,subject,level,price,subscribers,reviews,lectures,duration
0,1,2015-07-10,Business Finance,Beginner Level,1.7,56,8,18,2.000000
1,2,2016-10-25,Business Finance,All Levels,17.4,336,6,13,0.683333
2,3,2015-06-16,Business Finance,All Levels,8.3,871,79,59,2.000000


## 1. 결제 금액이 1억 이상이면서 구독자의 리뷰 작성 비율이 10% 이상인 교육의 수
- 결제 금액 = 강좌 가격 * 구독자 수
- 리뷰 작성 비율 = 리뷰 개수 / 구독자 수

In [88]:
df14_q1 = df14.copy()

df14_q1['amt'] = df14_q1['price'] * df14_q1['subscribers']
df14_q1['rate'] = df14_q1['reviews'] / df14_q1['subscribers']

In [194]:
cond1 = (df14_q1['amt'] >= int(1e4))
cond2 = (df14_q1['rate'] >= 0.1)

len(df14_q1[cond1 & cond2])

59

In [195]:
sum(cond1 & cond2)

59

### 답
59

## 2. 강좌 가격과 구독자 수의 피어슨 상관관계
- 2016년 개설된 Web Development 강좌 대상
- 반올림하여 소수점 둘째 자리까지

In [197]:
df14_q2 = df14.copy()
df14_q2['year'] = pd.to_datetime(df14_q2['published']).dt.year
df14_q2.head(3)

,id,published,subject,level,price,subscribers,reviews,lectures,duration,year
0,1,2015-07-10,Business Finance,Beginner Level,1.7,56,8,18,2.000000,2015
1,2,2016-10-25,Business Finance,All Levels,17.4,336,6,13,0.683333,2016
2,3,2015-06-16,Business Finance,All Levels,8.3,871,79,59,2.000000,2015


In [198]:
df14_q2 = df14_q2[(df14_q2['subject'] == 'Web Development') & (df14_q2['year'] == 2016)]
df14_q2.head(3)

,id,published,subject,level,price,subscribers,reviews,lectures,duration,year
6,7,2016-10-24,Web Development,Intermediate Level,17.4,9773,117,65,4.5,2016
36,37,2016-04-05,Web Development,All Levels,17.0,12068,81,12,1.5,2016
43,44,2016-03-28,Web Development,All Levels,2.2,3133,351,63,6.0,2016


In [199]:
df14_q2[['price', 'subscribers']].corr(method='pearson')

,price,subscribers
price,1.000000,0.034392
subscribers,0.034392,1.000000


### 답
0.03

## 3. 리뷰 작성 비율의 평균이 강좌 개설 년도별 차이가 있는지 일원 분산 분석
- 검정통계량은 반올림하여 소수점 첫째 자리까지

In [202]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

df14_q1['year'] = pd.to_datetime(df14_q1['published']).dt.year
formula = 'rate ~ C(year)'
anova = ols(formula, df14_q1).fit()

In [203]:
anova_lm(anova)

,df,sum_sq,mean_sq,F,PR(>F)
C(year),2.0,0.365685,0.182842,18.542038,1.049908e-08
Residual,1997.0,19.692334,0.009861,NaN,NaN


### 답
18.5

# 문제 15 유형
- Dataset_15_Mart_POS.csv 이용

In [100]:
import pandas as pd

df15_mart = pd.read_csv('datasets/Dataset_15_Mart_POS.csv', encoding='utf8')
df15_mart.head(3)

,Member_number,Date,itemDescription
0,1220,2015-01-01,canned beer
1,1220,2015-01-01,margarine
2,1220,2015-01-01,chocolate


## 1. 가장 많은 제품이 팔린 날짜에 가장 많이 팔린 제품의 판매 개수

In [106]:
df15_mart_date = df15_mart.groupby('Date').size()
df15_mart_date.sort_values(ascending=False)

Date
2015-01-21    96
2015-07-21    93
2015-11-29    92
2015-08-08    92
2015-04-30    91
              ..
2015-12-21    29
2015-11-05    29
2015-03-16    23
2015-03-17    23
2015-09-01    22
Length: 364, dtype: int64

In [108]:
date = df15_mart_date.idxmax()
date

'2015-01-21'

In [110]:
df15_mart_date = df15_mart[df15_mart['Date']==date]
df15_mart_date.groupby(['itemDescription']).size().sort_values(ascending=False)

itemDescription
soda                         7
tropical fruit               5
shopping bags                4
other vegetables             4
rolls/buns                   4
root vegetables              3
grapes                       3
frankfurter                  3
chicken                      3
whole milk                   3
beef                         3
bottled water                2
salty snack                  2
napkins                      2
pork                         2
bottled beer                 2
brown bread                  2
oil                          2
butter milk                  2
specialty bar                2
sausage                      1
white wine                   1
onions                       1
sugar                        1
rice                         1
pot plants                   1
sparkling wine               1
pip fruit                    1
photo/film                   1
pastry                       1
packaged fruit/vegetables    1
specialty chocolate    

### 답
7

## 2. 독립 2표본 t 검정
- 금, 토요일의 일별 주류제품 구매 제품 수 평균과 다른 요일의 일별 주류 제품 구매 제품 수 평균이 다른지 비교
- p-value를 반올림하여 소수점 둘째 자리까지
- 등분산 가정을 만족하지 않는다는 조건
- 1분기 (1월~3월) 데이터만 사용

In [111]:
df15_item = pd.read_csv('datasets/Dataset_15_item_list.csv', encoding='utf8')
df15_item.head(3)

,prod_id,prod_nm,alcohol,frozen
0,1,tropical fruit,0,0
1,2,whole milk,0,0
2,3,pip fruit,0,0


In [114]:
alcohols = df15_item[df15_item['alcohol'] == 1]['prod_nm'].values
alcohols

array(['canned beer', 'red/blush wine', 'sparkling wine', 'bottled beer',
       'white wine', 'liquor', 'liquor (appetizer)', 'rum', 'whisky',
       'liqueur'], dtype=object)

In [132]:
year = pd.to_datetime(df15_mart['Date']).dt.year
month = pd.to_datetime(df15_mart['Date']).dt.month
day = pd.to_datetime(df15_mart['Date']).dt.day
name = pd.to_datetime(df15_mart['Date']).dt.day_name()

In [133]:
name

0         Thursday
1         Thursday
2         Thursday
3         Thursday
4         Thursday
           ...    
20483    Wednesday
20484    Wednesday
20485    Wednesday
20486    Wednesday
20487    Wednesday
Name: Date, Length: 20488, dtype: object

In [134]:
df15_mart['name'] = name

In [135]:
df15_mart['month'] = month
df15_mart.head(3)

,Member_number,Date,itemDescription,month,name
0,1220,2015-01-01,canned beer,1,Thursday
1,1220,2015-01-01,margarine,1,Thursday
2,1220,2015-01-01,chocolate,1,Thursday


In [139]:
df15_merge = pd.merge(df15_mart, df15_item,
                      left_on='itemDescription', right_on='prod_nm',
                     how='left')
df15_merge.head(3)

,Member_number,Date,itemDescription,month,name,prod_id,prod_nm,alcohol,frozen
0,1220,2015-01-01,canned beer,1,Thursday,25,canned beer,1,0
1,1220,2015-01-01,margarine,1,Thursday,70,margarine,0,0
2,1220,2015-01-01,chocolate,1,Thursday,14,chocolate,0,0


In [174]:
df15_merge['fs'] = df15_merge['name'].apply(lambda x:1 if x in fs else 0)
df15_merge.head(3)

,Member_number,Date,itemDescription,month,name,prod_id,prod_nm,alcohol,frozen,fs
0,1220,2015-01-01,canned beer,1,Thursday,25,canned beer,1,0,0
1,1220,2015-01-01,margarine,1,Thursday,70,margarine,0,0,0
2,1220,2015-01-01,chocolate,1,Thursday,14,chocolate,0,0,0


In [181]:
df15_al = df15_merge[df15_merge['month'].isin([1, 2, 3])]
df15_al.head(3)

,Member_number,Date,itemDescription,month,name,prod_id,prod_nm,alcohol,frozen,fs
0,1220,2015-01-01,canned beer,1,Thursday,25,canned beer,1,0,0
1,1220,2015-01-01,margarine,1,Thursday,70,margarine,0,0,0
2,1220,2015-01-01,chocolate,1,Thursday,14,chocolate,0,0,0


In [182]:
len(df15_al)

5079

In [183]:
pt = pd.pivot_table(data=df15_al, columns='fs', index='Date', values='alcohol', aggfunc=sum)
pt

fs,0,1
Date,,
2015-01-01,4.0,NaN
2015-01-02,NaN,3.0
2015-01-03,NaN,2.0
2015-01-04,4.0,NaN
2015-01-05,5.0,NaN
...,...,...
2015-03-27,NaN,5.0
2015-03-28,NaN,2.0
2015-03-29,5.0,NaN


In [185]:
t, p = ttest_ind(pt[0].dropna(), pt[1].dropna(), equal_var=False)
print(t, p)

2.335264239960428 0.023062611047582393


### 답
0.02

## 3.

In [160]:
top10 = df15_mart['itemDescription'].value_counts().nlargest(10).index

In [161]:
top10

Index(['whole milk', 'other vegetables', 'rolls/buns', 'sausage', 'soda',
       'yogurt', 'tropical fruit', 'root vegetables', 'citrus fruit',
       'frankfurter'],
      dtype='object')

In [163]:
df15_top10 = df15_mart[df15_mart['itemDescription'].isin(top10)]
df15_top10.head(3)

,Member_number,Date,itemDescription,month,name
3,1235,2015-01-01,sausage,1,Thursday
4,1235,2015-01-01,sausage,1,Thursday
7,1422,2015-01-01,tropical fruit,1,Thursday


In [165]:
df15_pt = pd.pivot_table(data=df15_top10,
                        index=['Date', 'name'], values='itemDescription', aggfunc='count')
df15_pt

,,itemDescription
Date,name,
2015-01-01,Thursday,16
2015-01-02,Friday,13
2015-01-03,Saturday,28
2015-01-04,Sunday,21
2015-01-05,Monday,30
...,...,...
2015-12-26,Saturday,11
2015-12-27,Sunday,31
2015-12-28,Monday,21


In [166]:
df15_pt = df15_pt.reset_index()
df15_pt

,Date,name,itemDescription
0,2015-01-01,Thursday,16
1,2015-01-02,Friday,13
2,2015-01-03,Saturday,28
3,2015-01-04,Sunday,21
4,2015-01-05,Monday,30
...,...,...,...
359,2015-12-26,Saturday,11
360,2015-12-27,Sunday,31
361,2015-12-28,Monday,21
362,2015-12-29,Tuesday,27


In [167]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

formula = 'itemDescription ~ C(name)'

anova = ols(formula, df15_pt).fit()

In [168]:
anova_lm(anova)

,df,sum_sq,mean_sq,F,PR(>F)
C(name),6.0,219.527473,36.587912,0.86853,0.518128
Residual,357.0,15039.076923,42.126266,NaN,NaN


### 답
0.52

In [204]:
df15_day = df15_top10.groupby(['Date', 'name']).size()

In [205]:
df15_day.reset_index()

,Date,name,0
0,2015-01-01,Thursday,16
1,2015-01-02,Friday,13
2,2015-01-03,Saturday,28
3,2015-01-04,Sunday,21
4,2015-01-05,Monday,30
...,...,...,...
359,2015-12-26,Saturday,11
360,2015-12-27,Sunday,31
361,2015-12-28,Monday,21
362,2015-12-29,Tuesday,27
